# Import Libraries

In [1]:
from swarmintelligence import ParticleSwarmOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/PSO/optimal_params.pkl', 'rb'))
optimal_params

{'particleSize': 25,
 'maxIteration': 100,
 'phi1': 2.6,
 'phi2': 1.7,
 'inertia': 0.5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = ParticleSwarmOptimization(
                        k=threshold,
                        particleSize=optimal_params['particleSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        phi1=optimal_params['phi1'],
                        phi2=optimal_params['phi2'],
                        inertia=optimal_params['inertia']
                    )
                    gwo_g = ParticleSwarmOptimization(
                        k=threshold,
                        particleSize=optimal_params['particleSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        phi1=optimal_params['phi1'],
                        phi2=optimal_params['phi2'],
                        inertia=optimal_params['inertia']
                    )
                    gwo_b = ParticleSwarmOptimization(
                        k=threshold,
                        particleSize=optimal_params['particleSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        phi1=optimal_params['phi1'],
                        phi2=optimal_params['phi2'],
                        inertia=optimal_params['inertia']
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
930.7811479817464  |  2436.3584799702053  |  1787.831388216658
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.8215619288128
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7047355196313  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7811479817464  |  2436.3584799702053  |  1787.9236630530743
930.7683528911109  |  2436.3584799702053  |  1787.8950628016757
930.7683528911109  |  2436.3584799702053  |  1787.9236630530743
930.7047355196313  |  2436.3584799702053  |  1787.8888109611032
930.7811479817464  |  2436.3584799702053  |  1787.8045911666616
930.7811479817464  |  2436.3584799702053  |  1787.8888109611032
930.7811479817464  |  2436.3584799702053  |  1787.8888109611032
930.7811479817464  |  2436.3584799702053  |  1787.8950628016757
930.7

1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.813810795207
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.088716166724
1004.9139322682681  |  2393.647254097627  |  2181.813810795207
1004.9139322682681  |  2393.647254097627  |  2181.076154109842
1004.9139322682681  |  2393.647254097627  |  2181.813810795207
1004.9139322682681  |  2393.647254097627  |  2181.758352748785
1004.9139322682681  |  2393.647254097627  |  2181.782175541647
1004.9139322682681  |  2393.647254097627  |  2181.782175541647
1004.9139322682681  |  2393.647254097627  |  2181.8670821363294
1004.9139322682681  |  2393.647254097627  |  2181.758352748785
1004.9139322682681  |  2393.647254097627  |  2181.782175541647
1004.9139322682681  |  2393.647254097627  |  2180.9038541370096
1004.9139322682681  |  2393.647254097627  |  2181.6249270312555
1004.9139322682681  |  2393.647254097627  |  2181.

2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  2947.3859621212478  |  2947.3859621212478
2947.3859621212478  |  29

3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2513.3911207599995
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8307253277617  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2640.869259570156
3359.8652306428266  |  2004.6677233657592  |  2640.869259570156
3359.8652306428266  |  2004.6997362709596  |  2522.31841441529
3359.8652306428266  |  2004.699736270959

1656.2011866877924  |  5203.867083559779  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.867083559779  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.867083559779  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.927265675553  |  1759.2175161838823
1656.2011866877924  |  5203.867083559779  |  1759.2175161838823
1656.2011866877924  |  5203.867083559779

5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5224.097414725924  |  1589.872409058858
5640.462004216223  |  5636.585936244663  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5638.726718312159  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462004216223  |  5636.631173704616  |  1589.872409058858
5640.462

## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[121, 178]","[94, 166]","[125, 182]",930.762019,2436.207448,1787.898267,...,"[[125, 255, 255, 255, 255, 255, 255, 255, 255,...","[930.7811479817464, 930.7811479817464, 930.781...","[2436.3584799702053, 2436.3584799702053, 2436....","[1787.831388216658, 1787.9236630530743, 1787.8...","[4.2473461627960205, 3.681356906890869, 3.7497...","[2289.032999674479, 2293.7582194010415, 2300.5...","[47.843839725449286, 47.893195961441556, 47.96...","[14.534283071740466, 14.525327230279872, 14.51...","[0.7801771649646593, 0.7809698970724323, 0.781...","[0.9410097380078657, 0.941043257168062, 0.9409..."
1,AirplaneF16.tiff,3,otsu,max,"[107, 158, 196]","[77, 137, 188]","[123, 173, 202]",970.840058,2533.904633,1856.065221,...,"[[123, 255, 202, 255, 255, 202, 202, 202, 202,...","[970.612150105291, 970.8466210098144, 970.8591...","[2534.075361427361, 2534.062306243562, 2534.07...","[1855.71312021488, 1856.149138402158, 1856.328...","[4.101463556289673, 3.2542061805725098, 3.4485...","[1535.043062845866, 1563.9362462361653, 1571.7...","[39.17962560880165, 39.54663381675064, 39.6457...","[16.26959797540694, 16.18861315770791, 16.1668...","[0.7305593956507269, 0.7345485140931193, 0.737...","[0.9589120087272344, 0.9586027277811414, 0.958..."
2,AirplaneF16.tiff,4,otsu,max,"[84, 126, 169, 200]","[56, 109, 160, 199]","[113, 153, 181, 204]",993.831907,2578.355745,1900.109447,...,"[[113, 204, 204, 255, 204, 204, 204, 204, 204,...","[994.325085154997, 992.9156499282606, 993.7336...","[2587.611576470959, 2587.4995087126526, 2587.3...","[1900.0981001462817, 1900.5538024335615, 1896....","[4.300527811050415, 3.3547654151916504, 3.3562...","[1184.2523142496746, 1208.0363222757976, 1160....","[34.4129672398309, 34.75681691806368, 34.07057...","[17.396361187747505, 17.31000368365249, 17.483...","[0.716194599050584, 0.7186121296102325, 0.7165...","[0.9689632812331105, 0.9685503658534823, 0.969..."
3,AirplaneF16.tiff,5,otsu,max,"[77, 116, 152, 183, 203]","[49, 95, 136, 176, 204]","[112, 147, 174, 195, 208]",1004.898246,2611.413981,1921.447319,...,"[[112, 208, 208, 208, 208, 208, 208, 195, 208,...","[1005.1431780257418, 1004.7291424902639, 1005....","[2618.974892558141, 2588.1486693460565, 2611.7...","[1920.0411511363673, 1922.1282622241665, 1922....","[4.306756973266602, 3.421720504760742, 3.45159...","[883.5109481811523, 959.7839444478353, 827.610...","[29.723912060513708, 30.98037999198582, 28.768...","[18.66868425355403, 18.309068802184033, 18.952...","[0.726706046132859, 0.7333140062628511, 0.7272...","[0.9765828786362274, 0.9740155989094257, 0.976..."
4,Lena.png,2,otsu,max,"[137, 196]","[78, 146]","[96, 141]",1004.913932,2393.647254,2181.629353,...,"[[141, 141, 141, 141, 141, 141, 141, 141, 141,...","[1004.9139322682681, 1004.9139322682681, 1004....","[2393.647254097627, 2393.647254097627, 2393.64...","[2181.8670821363294, 2181.813810795207, 2181.8...","[4.018704891204834, 3.4118220806121826, 3.3956...","[1858.9784520467122, 1861.9157320658367, 1858....","[43.115872391112674, 43.14992157658965, 43.115...","[15.43806005098941, 15.431203393928147, 15.438...","[0.6394589731037388, 0.6398948445609046, 0.639...","[0.8954769944582632, 0.89554817486353, 0.89547..."
5,Lena.png,3,otsu,max,"[127, 174, 211]","[57, 107, 160]","[83, 111, 148]",1069.909798,2597.968218,2269.323064,...,"[[148, 148, 148, 148, 148, 148, 148, 148, 148,...","[1069.8535602787606, 1070.069410358829, 1069.8...","[2597.9819871813006, 2597.9819871813006, 2597....","[2269.941659588655, 2269.861831915663, 2268.89...","[4.20307183265686, 3.563401699066162, 3.346403...","[1196.4359957377117, 1179.0638122558594, 1162....","[34.589535928336936, 34.33749863131936, 34.092...","[17.351908902138323, 17.41543050632324, 17.477...","[0.6979372

# Save results

In [9]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/PSO/PSO_otsu_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/PSO/PSO_otsu_RGB_dict.pkl', 'wb'))